# Convolutional Neural Network
In this tutorial, we will implement convolutional neural network to classify mnist dataset.

In [22]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from ops import conv2d, linear


mnist = input_data.read_data_sets("mnist/", one_hot=True)
x_train = mnist.train.images.reshape(-1, 28, 28, 1)
y_train = mnist.train.labels
x_test = mnist.test.images.reshape(-1, 28, 28, 1)
y_test = mnist.test.labels

print "x_train: ", x_train.shape
print "y_train: ", y_train.shape
print "x_test: ", x_test.shape
print "y_test: ", y_test.shape

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
x_train:  (55000, 28, 28, 1)
y_train:  (55000, 10)
x_test:  (10000, 28, 28, 1)
y_test:  (10000, 10)


## Parameter Settings 

In [19]:
num_class = 10

params = {
    # weights and biases for convolution layer
    'w_conv1': tf.Variable(tf.random_normal([5, 5, 1, 32], stddev=0.1)),
    'b_conv1': tf.Variable(tf.zeros([32])),
    'w_conv2': tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.1)),
    'b_conv2': tf.Variable(tf.zeros([64])),
    
    # weights and biases for fully-connected layer
    'w_h': tf.Variable(tf.random_normal([7*7*64, 128], stddev=0.1)),
    'b_h': tf.Variable(tf.zeros([128])),
    'w_out': tf.Variable(tf.random_normal([128, num_class], stddev=0.1)),
    'b_out': tf.Variable(tf.zeros([num_class]))
}

## Define and Construct Model

In [20]:
def convolutional_network(x):
    # 1st convolution layer
    conv1 = tf.nn.conv2d(x, params['w_conv1'], strides=[1,1,1,1], padding='SAME') + params['b_conv1']
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # 2nd convolution layer
    conv2 = tf.nn.conv2d(conv1, params['w_conv2'], strides=[1,1,1,1], padding='SAME') + params['b_conv2']
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # fully connected layer
    conv2 = tf.reshape(conv2, [tf.shape(conv2)[0], -1])
    h = tf.matmul(conv2, params['w_h']) + params['b_h']
    h = tf.nn.relu(h)
    out = h = tf.matmul(h, params['w_out']) + params['b_out']
    
    return out

In [29]:
# construct model
x = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 1])
y = tf.placeholder(dtype=tf.int64, shape=[None, 10])
out = convolutional_network(x)

# define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))
train_op = tf.train.RMSPropOptimizer(learning_rate=0.01).minimize(loss)

# find correct prediction and accuracy
correct_pred = tf.equal(tf.argmax(out,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [31]:
batch_size = 100
num_epoch = 15
num_iter_per_epoch = int(x_train.shape[0] / batch_size)

# launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # initialize tensor variables
    tf.initialize_all_variables().run()
    
    # training cycle
    for epoch in range(num_epoch):
        avg_loss = 0.
        # loop over all batches
        for i in range(num_iter_per_epoch):
            x_batch = x_train[i*batch_size:(i+1)*batch_size]
            y_batch = y_train[i*batch_size:(i+1)*batch_size]
        
            # run optimization op (backprop) and loss op (to get loss value)
            _, c = sess.run([train_op, loss], feed_dict={x: x_batch, y: y_batch})
            
            # compute average loss
            avg_loss += c / num_iter_per_epoch
        print "Epoch %d, Loss: %.3f"% (epoch+1, avg_loss)
    print "Finished training!\n"
    
    
    num_iter_per_epoch = int(x_test.shape[0] / batch_size)
    for i in range(num_iter_per_epoch):
        x_batch = x_test[i*batch_size:(i+1)*batch_size]
        y_batch = y_test[i*batch_size:(i+1)*batch_size]
       
        sess.run(accuracy, feed_dict={x: x_batch, y: y_batch})
    
    
    
    print "Test accuracy:", sess.run(accuracy, {x: mnist.test.images, y: mnist.test.labels})

Epoch 1, Loss: 0.324
Epoch 2, Loss: 0.085
Epoch 3, Loss: 0.071
Epoch 4, Loss: 0.063
Epoch 5, Loss: 0.061
Epoch 6, Loss: 0.059
Epoch 7, Loss: 0.059
Epoch 8, Loss: 0.061
Epoch 9, Loss: 0.066
Epoch 10, Loss: 0.060
Epoch 11, Loss: 0.055
Epoch 12, Loss: 0.059
Epoch 13, Loss: 0.056
Epoch 14, Loss: 0.061
Epoch 15, Loss: 0.058
Finished training!

Test accuracy:

ValueError: Cannot feed value of shape (10000, 784) for Tensor u'Placeholder_5:0', which has shape '(?, 28, 28, 1)'